# Train Pix2Pix model for image-to-image translation

make sure you have done the dataset preparation before running this notebook. 

In this session, we will have 2 parts:
- Baseline Training
- Enhanced Training

# Install

In [ ]:
import os

# check if `pytorch-CycleGAN-and-pix2pix` is already cloned
if not os.path.exists('pytorch-CycleGAN-and-pix2pix'):
	!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

In [ ]:
os.chdir('pytorch-CycleGAN-and-pix2pix/')

In [ ]:
!pwd

In [ ]:
%pip install -q -r requirements.txt

# Basline model (train the road river in one model)

## Datasets

Put the dataset in the `pytorch-CycleGAN-and-pix2pix/datasets` folder

(We have finished this part at the previous step when running `preprocess_dataset.ipynb`)

Each dataset should have the following directory structure:

```
datasets
└── training_dataset
	├── train
	├── test
```

## Training (Baseline)

official sample code: 

`python train.py --dataroot ./datasets/facades --name facades_pix2pix --model pix2pix --direction AtoB --display_id 0`

- Change the `--dataroot` and `--name` to your own dataset's path and desired model's name. 

- `--model pix2pix` specifies the model type we want to train (the other is `cycle_gan`)

- `--direction AtoB` (convert from domain A to domain B)

- `--n_epochs 100` and `--n_epochs_decay 100` to train for 200 epochs(100 + 100)

- `--display_id 0` to disable visdom display

check `options/train_options.py` for more arguments options

## some tips for enhancing :
[reference](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix/issues/760)
- `--netG resnet_9blocks` to adapt the image size
- `--preprocess none` to disable preprocessing

In [ ]:
# the below code train the full dataset in one stage (in which the model is trained on both the road and river images)
# use this line as baseline
! python train.py --dataroot ./datasets/training_dataset/ --name ROAD_RIVER_pix2pix --model pix2pix --direction AtoB --display_id 0

# maybe better performance
# ! python train.py --dataroot ./datasets/training_dataset/ --name ROAD_RIVER_pix2pix --model pix2pix --direction AtoB --display_id 0 --netG resnet_9blocks --preprocess none 

# use nohup to run the training in the background
# ! nohup python train.py ... > train.log &

## Results
the results will be saved in `pytorch-CycleGAN-and-pix2pix/checkpoints/ROAD_RIVER_pix2pix` folder

- `/web/index.html` for the visualization of the results
- `latest_net_G.pth` for the latest model

# Enhanced (train 2 domain-specific models)
- one for RIVER
- one for ROAD

## Datasets

Put the dataset in the `pytorch-CycleGAN-and-pix2pix/datasets` folder

(We have finished this part at the previous step when running `preprocess_dataset.ipynb`)

Each dataset should have the following directory structure:

```
datasets
└── train_ROAD
    ├── train
    ├── test
└── train_RIVER
    ├── train
    ├── test
```

## Training (Enhanced)
Add argument:
- `--n_epochs=200 (default 100)`
- `--n_epochs_decay=200 (default 100)`


In [ ]:
# add the n_epochs and n_epochs_decay parameters up to total 400 epochs for each model
! python train.py --dataroot ./datasets/train_ROAD --name ROAD_pix2pix --model pix2pix --direction AtoB --n_epochs 500 --n_epochs_decay 100 --display_id 0 --epoch_count 401 --continue_train
! python train.py --dataroot ./datasets/train_RIVER --name RIVER_pix2pix --model pix2pix --direction AtoB --n_epochs 500 --n_epochs_decay 100 --display_id 0 --epoch_count 401 --continue_train

# use nohup to run the training in the background
# ! nohup python train.py ... > road.log &	
# ! nohup python train.py ... > river.log &

## Results
after training, you can find the results in 
- `pytorch-CycleGAN-and-pix2pix/checkpoints/ROAD_pix2pix` folder
- `pytorch-CycleGAN-and-pix2pix/checkpoints/RIVER_pix2pix` folder

Each folder contains:
- `/web/index.html` for the visualization of the results
- `latest_net_G.pth` for the latest model